In [1]:
import requests
import json
import time
import pprint as pp

In [2]:
def fetch_abi(contract_addr):
    ABI_ENDPOINT = 'https://api.etherscan.io/api?module=contract&action=getabi&address='
    response = requests.get('%s%s'%(ABI_ENDPOINT, contract_addr))
    response_json = response.json()
    abi_json = json.loads(response_json['result'])
    return json.dumps(abi_json)


In [3]:
sql_code = """
-- SELECT * from `bigquery-public-data.crypto_ethereum.transactions` as tx
-- where tx.hash = '0x56f2ce34e4b20578742ed8ddc9fcbacaec62d477a530dff7ace8da2fe64b1208'


SELECT * FROM `bigquery-public-data.crypto_ethereum.logs` 
where transaction_hash='0x56f2ce34e4b20578742ed8ddc9fcbacaec62d477a530dff7ace8da2fe64b1208'

"""

In [4]:
TX_HASH = '0x56f2ce34e4b20578742ed8ddc9fcbacaec62d477a530dff7ace8da2fe64b1208'
OPENSEA_CONTRACT_ADDR = "0x7be8076f4ea4a4ad08075c2508e481d6c946d12b"
NFT_CONTRACT_ADDR = "0x08dfdbb07f013856d0d18592d20185d64c9c4ef0"

BORED_APE_ADDR = "0xBC4CA0EdA7647A8aB7C2061c2E118A18a936f13D"

In [5]:
opensea_abi = fetch_abi(OPENSEA_CONTRACT_ADDR)
time.sleep(5)
nft_abi = fetch_abi(BORED_APE_ADDR)
pp.pprint(nft_abi)

('[{"inputs": [{"internalType": "string", "name": "name", "type": "string"}, '
 '{"internalType": "string", "name": "symbol", "type": "string"}, '
 '{"internalType": "uint256", "name": "maxNftSupply", "type": "uint256"}, '
 '{"internalType": "uint256", "name": "saleStart", "type": "uint256"}], '
 '"stateMutability": "nonpayable", "type": "constructor"}, {"anonymous": '
 'false, "inputs": [{"indexed": true, "internalType": "address", "name": '
 '"owner", "type": "address"}, {"indexed": true, "internalType": "address", '
 '"name": "approved", "type": "address"}, {"indexed": true, "internalType": '
 '"uint256", "name": "tokenId", "type": "uint256"}], "name": "Approval", '
 '"type": "event"}, {"anonymous": false, "inputs": [{"indexed": true, '
 '"internalType": "address", "name": "owner", "type": "address"}, {"indexed": '
 'true, "internalType": "address", "name": "operator", "type": "address"}, '
 '{"indexed": false, "internalType": "bool", "name": "approved", "type": '
 '"bool"}], "name"

In [9]:
opensea_abi = fetch_abi(OPENSEA_CONTRACT_ADDR)
time.sleep(5)
nft_abi = fetch_abi(NFT_CONTRACT_ADDR)
pp.pprint(nft_abi)

('[{"inputs": [{"internalType": "address", "name": "_withdrawAddress", "type": '
 '"address"}, {"internalType": "string", "name": "_uri", "type": "string"}], '
 '"stateMutability": "nonpayable", "type": "constructor"}, {"anonymous": '
 'false, "inputs": [{"indexed": true, "internalType": "address", "name": '
 '"owner", "type": "address"}, {"indexed": true, "internalType": "address", '
 '"name": "approved", "type": "address"}, {"indexed": true, "internalType": '
 '"uint256", "name": "tokenId", "type": "uint256"}], "name": "Approval", '
 '"type": "event"}, {"anonymous": false, "inputs": [{"indexed": true, '
 '"internalType": "address", "name": "owner", "type": "address"}, {"indexed": '
 'true, "internalType": "address", "name": "operator", "type": "address"}, '
 '{"indexed": false, "internalType": "bool", "name": "approved", "type": '
 '"bool"}], "name": "ApprovalForAll", "type": "event"}, {"anonymous": false, '
 '"inputs": [{"indexed": true, "internalType": "uint256", "name": "_amount",

In [10]:
import traceback
import sys
from functools import lru_cache
from web3 import Web3
from web3.auto import w3
from web3.contract import Contract
from web3._utils.events import get_event_data
from web3._utils.abi import exclude_indexed_event_inputs, get_abi_input_names, get_indexed_event_inputs, normalize_event_input_types
from web3.exceptions import MismatchedABI, LogTopicError
from web3.types import ABIEvent
from eth_utils import event_abi_to_log_topic, to_hex
from hexbytes import HexBytes

import json
import re


def decode_tuple(t, target_field):
    output = dict()
    for i in range(len(t)):
        if isinstance(t[i], (bytes, bytearray)):
            output[target_field[i]['name']] = to_hex(t[i])
        elif isinstance(t[i], (tuple)):
            output[target_field[i]['name']] = decode_tuple(t[i], target_field[i]['components'])
        else:
            output[target_field[i]['name']] = t[i]
    return output


def decode_list_tuple(l, target_field):
    output = l
    for i in range(len(l)):
        output[i] = decode_tuple(l[i], target_field)
    return output


def decode_list(l):
    output = l
    for i in range(len(l)):
        if isinstance(l[i], (bytes, bytearray)):
            output[i] = to_hex(l[i])
        else:
            output[i] = l[i]
    return output


def convert_to_hex(arg, target_schema):
    output = dict()
    for k in arg:
        if isinstance(arg[k], (bytes, bytearray)):
            output[k] = to_hex(arg[k])
        elif isinstance(arg[k], (list)) and len(arg[k]) > 0:
            target = [a for a in target_schema if 'name' in a and a['name'] == k][0]
            if target['type'] == 'tuple[]':
                target_field = target['components']
                output[k] = decode_list_tuple(arg[k], target_field)
            else:
                output[k] = decode_list(arg[k])
        elif isinstance(arg[k], (tuple)):
            target_field = [a['components'] for a in target_schema if 'name' in a and a['name'] == k][0]
            output[k] = decode_tuple(arg[k], target_field)
        else:
            output[k] = arg[k]
    return output

@lru_cache(maxsize=None)
def _get_contract(address, abi):
    """
    This helps speed up execution of decoding across a large dataset by caching the contract object
    It assumes that we are decoding a small set, on the order of thousands, of target smart contracts
    """
    if isinstance(abi, (str)):
        abi = json.loads(abi)

    contract = w3.eth.contract(address=Web3.toChecksumAddress(address), abi=abi)
    return (contract, abi)

def decode_tx(address, input_data, abi):
    if abi is not None:
        try:
            (contract, abi) = _get_contract(address, abi)
            func_obj, func_params = contract.decode_function_input(input_data)
            target_schema = [a['inputs'] for a in abi if 'name' in a and a['name'] == func_obj.fn_name][0]
            decoded_func_params = convert_to_hex(func_params, target_schema)
            return (func_obj.fn_name, json.dumps(decoded_func_params), json.dumps(target_schema))
        except:
            e = sys.exc_info()[0]
            return ('decode error', repr(e), None)
    else:
        return ('no matching abi', None, None)


In [11]:
# decode bored ape tx
input_str = "0x23b872dd000000000000000000000000b9ea03e69e4983c4ab303cf3379864dc8fb9e462000000000000000000000000d9c65cd55626b6cb516cd9a594b0b1518ee8d02300000000000000000000000000000000000000000000000000000000000024bf"

output = decode_tx(
    BORED_APE_ADDR, 
    input_str,
    nft_abi)
pp.pprint(output)

('transferFrom',
 '{"from": "0xB9Ea03E69e4983c4Ab303cF3379864DC8FB9e462", "to": '
 '"0xd9c65cd55626B6cb516cd9a594B0B1518Ee8D023", "tokenId": 9407}',
 '[{"internalType": "address", "name": "from", "type": "address"}, '
 '{"internalType": "address", "name": "to", "type": "address"}, '
 '{"internalType": "uint256", "name": "tokenId", "type": "uint256"}]')


In [12]:
output = decode_tx(
    "0x7be8076f4ea4a4ad08075c2508e481d6c946d12b",
    "0xab834bab0000000000000000000000007be8076f4ea4a4ad08075c2508e481d6c946d12b00000000000000000000000065cf158c13b53627bd16466bf5ce799ee691f044000000000000000000000000271ae5a9e689ee106eef2e70861122aaf2a3135f000000000000000000000000000000000000000000000000000000000000000000000000000000000000000008dfdbb07f013856d0d18592d20185d64c9c4ef0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000007be8076f4ea4a4ad08075c2508e481d6c946d12b000000000000000000000000271ae5a9e689ee106eef2e70861122aaf2a3135f00000000000000000000000000000000000000000000000000000000000000000000000000000000000000005b3256965e7c3cf26e11fcaf296dfc8807c0107300000000000000000000000008dfdbb07f013856d0d18592d20185d64c9c4ef00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000002ee000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000004edec84a03800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000061d392310000000000000000000000000000000000000000000000000000000000000000caf1e4dfb4f63c9a48f8d7565f23653ae1f81181a0ed458b194227722b96bc9400000000000000000000000000000000000000000000000000000000000002ee000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000004edec84a03800000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000061d38eb20000000000000000000000000000000000000000000000000000000061d4e08b596593884f8708d710a17c4b984c163961909ded04b9f5058c59d88b66a4583f0000000000000000000000000000000000000000000000000000000000000001000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000100000000000000000000000000000000000000000000000000000000000000010000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006a0000000000000000000000000000000000000000000000000000000000000074000000000000000000000000000000000000000000000000000000000000007e0000000000000000000000000000000000000000000000000000000000000088000000000000000000000000000000000000000000000000000000000000009200000000000000000000000000000000000000000000000000000000000000940000000000000000000000000000000000000000000000000000000000000001c000000000000000000000000000000000000000000000000000000000000001c7392bfa8e11e8bb20e71ce821156757d41cfbd0651ec73f8296d1b8151f9f39d35856a14aa0037261ed79285bfcab16d0cc6d1b3c152d8847e8c92551008aea17392bfa8e11e8bb20e71ce821156757d41cfbd0651ec73f8296d1b8151f9f39d35856a14aa0037261ed79285bfcab16d0cc6d1b3c152d8847e8c92551008aea10000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006423b872dd000000000000000000000000000000000000000000000000000000000000000000000000000000000000000065cf158c13b53627bd16466bf5ce799ee691f0440000000000000000000000000000000000000000000000000000000000000b7400000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006423b872dd000000000000000000000000271ae5a9e689ee106eef2e70861122aaf2a3135f00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000b7400000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000006400000000ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000064000000000000000000000000000000000000000000000000000000000000000000000000ffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffffff00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000",
    opensea_abi,
)
pp.pprint(output)

('atomicMatch_',
 '{"addrs": ["0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b", '
 '"0x65cf158c13b53627Bd16466bF5CE799Ee691f044", '
 '"0x271ae5A9e689ee106EeF2E70861122Aaf2A3135f", '
 '"0x0000000000000000000000000000000000000000", '
 '"0x08dFdbb07f013856D0d18592D20185d64C9c4EF0", '
 '"0x0000000000000000000000000000000000000000", '
 '"0x0000000000000000000000000000000000000000", '
 '"0x7Be8076f4EA4A4AD08075C2508e481d6C946D12b", '
 '"0x271ae5A9e689ee106EeF2E70861122Aaf2A3135f", '
 '"0x0000000000000000000000000000000000000000", '
 '"0x5b3256965e7C3cF26E11FCAf296DfC8807C01073", '
 '"0x08dFdbb07f013856D0d18592D20185d64C9c4EF0", '
 '"0x0000000000000000000000000000000000000000", '
 '"0x0000000000000000000000000000000000000000"], "uints": [750, 0, 0, 0, '
 '22200000000000000, 0, 1641255473, 0, '
 '91794585185724084513338404068604283169366609595585469924337862820448645332116, '
 '750, 0, 0, 0, 22200000000000000, 0, 1641254578, 1641341067, '
 '404353133090942713539921500206760119256092957460296653368

In [13]:
from web3._utils.events import get_event_data

@lru_cache(maxsize=None)
def _get_topic2abi(abi):
    if isinstance(abi, (str)):
        abi = json.loads(abi)

    event_abi = [a for a in abi if a["type"] == "event"]
    topic2abi = {event_abi_to_log_topic(_): _ for _ in event_abi}
    return topic2abi


@lru_cache(maxsize=None)
def _get_hex_topic(t):
    hex_t = HexBytes(t)
    return hex_t


def decode_log(data, topics, abi):
    if abi is not None:
        try:
            topic2abi = _get_topic2abi(abi)

            log = {
                "address": None,  # Web3.toChecksumAddress(address),
                "blockHash": None,  # HexBytes(blockHash),
                "blockNumber": None,
                "data": data,
                "logIndex": None,
                "topics": [_get_hex_topic(_) for _ in topics],
                "transactionHash": None,  # HexBytes(transactionHash),
                "transactionIndex": None,
            }
            event_abi = topic2abi[log["topics"][0]]
            evt_name = event_abi["name"]

            data = get_event_data(w3.codec, event_abi, log)["args"]
            target_schema = event_abi["inputs"]
            decoded_data = convert_to_hex(data, target_schema)

            return (evt_name, json.dumps(decoded_data), json.dumps(target_schema))
        except Exception:
            return ("decode error", traceback.format_exc(), None)

    else:
        return ("no matching abi", None, None)

In [14]:
decode_log(
    "0x",
    [
        "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925",
        "0x000000000000000000000000271ae5a9e689ee106eef2e70861122aaf2a3135f",
        "0x0000000000000000000000000000000000000000000000000000000000000000",
        "0x0000000000000000000000000000000000000000000000000000000000000b74",
    ],
    nft_abi,
)

('Approval',
 '{"owner": "0x271ae5A9e689ee106EeF2E70861122Aaf2A3135f", "approved": "0x0000000000000000000000000000000000000000", "tokenId": 2932}',
 '[{"indexed": true, "internalType": "address", "name": "owner", "type": "address"}, {"indexed": true, "internalType": "address", "name": "approved", "type": "address"}, {"indexed": true, "internalType": "uint256", "name": "tokenId", "type": "uint256"}]')

In [15]:
decode_log('0x',
          ['0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef',
           '0x000000000000000000000000271ae5a9e689ee106eef2e70861122aaf2a3135f',
           '0x00000000000000000000000065cf158c13b53627bd16466bf5ce799ee691f044',
           '0x0000000000000000000000000000000000000000000000000000000000000b74'],
          nft_abi)

('Transfer',
 '{"from": "0x271ae5A9e689ee106EeF2E70861122Aaf2A3135f", "to": "0x65cf158c13b53627Bd16466bF5CE799Ee691f044", "tokenId": 2932}',
 '[{"indexed": true, "internalType": "address", "name": "from", "type": "address"}, {"indexed": true, "internalType": "address", "name": "to", "type": "address"}, {"indexed": true, "internalType": "uint256", "name": "tokenId", "type": "uint256"}]')

In [27]:
input_data = '0x0000000000000000000000000000000000000000000000000000000000000000dd25ddd1ce0820cd864016d0fe09690345b73198f4718dfe0ea3e8ed481f589500000000000000000000000000000000000000000000000001b236714472c000'

out = decode_log(input_data,
    ['0xc4109843e0b7d514e4c093114b863f8e7d8d9a458c372cd51bfe526b588006c9', 
            '0x000000000000000000000000e51341e05699ed92c4d0402f4e955862423d3aa8',
            '0x000000000000000000000000adc3c1ac7b171b5a2f7e58c52203c1af37e76ab4', 
            '0x0000000000000000000000000000000000000000000000000000000000000000'],
           opensea_abi
          )

In [26]:
base = 10**18 #1000000000000000000
122220000000000000 / base

0.12222

In [29]:
out[0]

'OrdersMatched'

In [30]:
out[1]

'{"maker": "0xE51341E05699Ed92C4d0402F4E955862423d3aa8", "taker": "0xAdc3C1aC7b171B5A2F7E58c52203c1aF37E76ab4", "metadata": "0x0000000000000000000000000000000000000000000000000000000000000000", "buyHash": "0x0000000000000000000000000000000000000000000000000000000000000000", "sellHash": "0xdd25ddd1ce0820cd864016d0fe09690345b73198f4718dfe0ea3e8ed481f5895", "price": 122220000000000000}'